# Use engineered features to train a model
This code will use the features created to train a logistic regression model.
The process is:
1. Test each feature individually to see their effectiveness.
2. Remove any features which corrolate highly with one another.
3. Order the features from most to least effective.
4. Test combinations of features by working down this list adding a new feature each time.
5. Use the combination with the highest TRP.

In [1]:
#Import libraries
import pandas as pd
import numpy as np
import re

In [2]:
#Import and combine prices files
path = "C:\\Users\\Robert\\Documents\\python_scripts\\stock_trading_ml_modelling\\historical_prices\\"
df_ft = pd.read_hdf(path + "all_hist_prices_w_ft_eng2.h5")
print("SHAPE: {}".format(df_ft.shape))
print(df_ft.dtypes)
df_ft.head()

SHAPE: (254719, 55)
Ticker                                 object
Date                           datetime64[ns]
Open                                  float64
Close                                 float64
High                                  float64
Low                                   float64
Volume                                float64
change_price                          float64
per_change_price                      float64
close_shift1                          float64
change_close_shift1                   float64
vol_shift1                            float64
change_vol_shift1                     float64
EMA26_shift1                          float64
change_EMA26_shift1                   float64
prev_max_close                        float64
prev_min_close                        float64
prev_max_EMA26                        float64
prev_min_EMA26                        float64
prev_max_close_date_change            float64
prev_min_close_date_change            float64
prev_max_MACD_

,Ticker,Date,Open,Close,High,Low,Volume,change_price,per_change_price,close_shift1,...,min_change_MACD_neg_val,max_change_EMA26_pos_val,max_change_EMA26_neg_val,min_change_EMA26_pos_val,min_change_EMA26_neg_val,max_change_signal_pos_val,max_change_signal_neg_val,min_change_signal_pos_val,min_change_signal_neg_val,signal
0,III,2007-12-31,0.866039,0.813036,0.855672,0.848469,0.053557,0.564065,0.476740,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,hold
1,III,2008-01-07,0.837258,0.771148,0.821356,0.799022,0.236390,0.524300,0.457015,0.842741,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,hold
2,III,2008-01-14,0.784406,0.747650,0.767864,0.760208,0.325624,0.605302,0.493657,0.799322,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,hold
3,III,2008-01-21,0.757195,0.764508,0.797134,0.724054,0.264801,0.730486,0.556505,0.774966,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,hold
4,III,2008-01-28,0.778127,0.808950,0.803189,0.783603,0.173403,0.799705,0.589990,0.792439,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,hold


In [3]:
df_ft.columns

Index(['Ticker', 'Date', 'Open', 'Close', 'High', 'Low', 'Volume',
       'change_price', 'per_change_price', 'close_shift1',
       'change_close_shift1', 'vol_shift1', 'change_vol_shift1',
       'EMA26_shift1', 'change_EMA26_shift1', 'prev_max_close',
       'prev_min_close', 'prev_max_EMA26', 'prev_min_EMA26',
       'prev_max_close_date_change', 'prev_min_close_date_change',
       'prev_max_MACD_date_change', 'prev_min_MACD_date_change',
       'prev_max_EMA26_date_change', 'prev_min_EMA26_date_change',
       'prev_max_signal_date_change', 'prev_min_signal_date_change',
       'close_13_norm', 'close_26_norm', 'close_52_norm', 'MACD_pos_val',
       'MACD_neg_val', 'signal_pos_val', 'signal_neg_val',
       'change_price_pos_val', 'change_price_neg_val',
       'per_change_price_pos_val', 'per_change_price_neg_val',
       'max_change_close_pos_val', 'max_change_close_neg_val',
       'min_change_close_pos_val', 'min_change_close_neg_val',
       'max_change_MACD_pos_val', 'max_

# Build the logistic regression model
This model is designed to predict if a week should be buy, hold or sell.


In [4]:
#Import the modules
from sklearn.linear_model import LogisticRegression as lr

In [5]:
feature_cols = [
    #NORMALISED COLS
    #Standard features
    "Open"
    ,"Close"
    ,"High"
    ,"Low"
    ,"Volume"
    ,"change_price"
    ,"per_change_price"
    #Shifted features
    ,"close_shift1"
    ,"change_close_shift1"
    ,"vol_shift1"
    ,"change_vol_shift1"
    ,"EMA26_shift1"
    ,"change_EMA26_shift1"
    #Prev max/min features
    ,"prev_max_close"
    ,"prev_min_close"
    ,"prev_max_EMA26"
    ,"prev_min_EMA26"
    #Date changes
    ,"prev_max_close_date_change"
    ,"prev_min_close_date_change"
    ,"prev_max_MACD_date_change"
    ,"prev_min_MACD_date_change"
    ,"prev_max_EMA26_date_change"
    ,"prev_min_EMA26_date_change"
    ,"prev_max_signal_date_change"
    ,"prev_min_signal_date_change"
    #Change to periodic max mins
    ,"close_13_norm"
    ,"close_26_norm"
    ,"close_52_norm"
    #Pos neg features
    ,"MACD_pos_val"
    ,"MACD_neg_val"
    ,"signal_pos_val"
    ,"signal_neg_val"
    ,"change_price_pos_val"
    ,"change_price_neg_val"
    ,"per_change_price_pos_val"
    ,"per_change_price_neg_val"
    ,"max_change_close_pos_val"
    ,"max_change_close_neg_val"
    ,"min_change_close_pos_val"
    ,"min_change_close_neg_val"
    ,"max_change_MACD_pos_val"
    ,"max_change_MACD_neg_val"
    ,"min_change_MACD_pos_val"
    ,"min_change_MACD_neg_val"
    ,"max_change_EMA26_pos_val"
    ,"max_change_EMA26_neg_val"
    ,"min_change_EMA26_pos_val"
    ,"min_change_EMA26_neg_val"
    ,"max_change_signal_pos_val"
    ,"max_change_signal_neg_val"
    ,"min_change_signal_pos_val"
    ,"min_change_signal_neg_val"  
]
cat_cols = [
    'signal'
]
df_model = df_ft[feature_cols+cat_cols].copy()
print("COLUMNS:",df_model.columns)
print("COLUMNS:",df_model.dtypes)
print("SHAPE:",df_model.shape)

COLUMNS: Index(['Open', 'Close', 'High', 'Low', 'Volume', 'change_price',
       'per_change_price', 'close_shift1', 'change_close_shift1', 'vol_shift1',
       'change_vol_shift1', 'EMA26_shift1', 'change_EMA26_shift1',
       'prev_max_close', 'prev_min_close', 'prev_max_EMA26', 'prev_min_EMA26',
       'prev_max_close_date_change', 'prev_min_close_date_change',
       'prev_max_MACD_date_change', 'prev_min_MACD_date_change',
       'prev_max_EMA26_date_change', 'prev_min_EMA26_date_change',
       'prev_max_signal_date_change', 'prev_min_signal_date_change',
       'close_13_norm', 'close_26_norm', 'close_52_norm', 'MACD_pos_val',
       'MACD_neg_val', 'signal_pos_val', 'signal_neg_val',
       'change_price_pos_val', 'change_price_neg_val',
       'per_change_price_pos_val', 'per_change_price_neg_val',
       'max_change_close_pos_val', 'max_change_close_neg_val',
       'min_change_close_pos_val', 'min_change_close_neg_val',
       'max_change_MACD_pos_val', 'max_change_MACD_neg_

In [6]:
#Remove rows with missing or infinate values
df_model.dropna(inplace=True)
df_model.reset_index(inplace=True)
print("COLUMNS:",df_model.columns)
print("SHAPE:",df_model.shape)

COLUMNS: Index(['index', 'Open', 'Close', 'High', 'Low', 'Volume', 'change_price',
       'per_change_price', 'close_shift1', 'change_close_shift1', 'vol_shift1',
       'change_vol_shift1', 'EMA26_shift1', 'change_EMA26_shift1',
       'prev_max_close', 'prev_min_close', 'prev_max_EMA26', 'prev_min_EMA26',
       'prev_max_close_date_change', 'prev_min_close_date_change',
       'prev_max_MACD_date_change', 'prev_min_MACD_date_change',
       'prev_max_EMA26_date_change', 'prev_min_EMA26_date_change',
       'prev_max_signal_date_change', 'prev_min_signal_date_change',
       'close_13_norm', 'close_26_norm', 'close_52_norm', 'MACD_pos_val',
       'MACD_neg_val', 'signal_pos_val', 'signal_neg_val',
       'change_price_pos_val', 'change_price_neg_val',
       'per_change_price_pos_val', 'per_change_price_neg_val',
       'max_change_close_pos_val', 'max_change_close_neg_val',
       'min_change_close_pos_val', 'min_change_close_neg_val',
       'max_change_MACD_pos_val', 'max_change_

In [7]:
#Shuffle the dataset
np.random.seed(0)
rand_index = np.random.permutation(df_model.index.values)
df_model_rand = df_model.iloc[rand_index]
df_model_rand.index.values

array([101124,  55124, 174918, ..., 173685,  43567, 199340], dtype=int64)

In [8]:
#Create the train and test dataset
ind_lim = int(np.round(len(df_model_rand.index)*0.7))
print("ind_lim: {}".format(ind_lim))

df_train = df_model_rand.iloc[:ind_lim].reset_index()
df_test = df_model_rand.iloc[ind_lim:].reset_index()

print("train len: {}".format(len(df_train)))
print("test len: {}".format(len(df_test)))

ind_lim: 162303
train len: 162303
test len: 69559


# Creating a multi-classification model
The model will take a one-vs-all approach (IE 1 if it is this value, 0 if it is anything else) using the variables of "buy", "hold", and "sell" individually and building a model to find the error rate on each one.

Error rate is determined by the four classifications:
- True positive - correct - model is 1, actual is 1
- True negative - correct - model is 0, actual is 0
- False positive - error - model is 1, actual is 0
- False negative - error - model is 0, actual is 1

In [92]:
#Function for building a model and outputting a dictionary of models created
def build_models(unique_classes,df_train,features):
    #Create a dictionary of models
    models = {}
    for cl in unique_classes:
        lm = lr() #create the model
        x_train = df_train[features]
        y_train = df_train["signal"] == cl
        lm.fit(x_train,y_train)
        models[cl] = lm
    return models

#Function for creating a dataframe with model probabilities and the most likely outcome
def calc_probs_df(models,unique_classes,df_test,features):
    modelled_probs = pd.DataFrame(columns=unique_classes)
    for cl in unique_classes:
        x_test = df_test[features]
        modelled_probs[cl] = models[cl].predict_proba(x_test)[:,1] 
        #Column index 1 as predict_proba outputs 2 columns,
            #the first is the probability that it is a negative result
            #the second is the probability that it is a positive result
            #that's why we want column index 1, we are looking at the positive result
    #Determine the most probable result
    modelled_probs["signal"] = modelled_probs.idxmax(axis=1)
    modelled_probs["signal_prob"] = modelled_probs[unique_classes].max(axis=1)
#     modelled_probs["bs_bool"] = (modelled_probs["buy"] >= 1/3) | (modelled_probs["sell"] >= 1/3)
#     modelled_probs.loc[modelled_probs["bs_bool"],"signal"] = modelled_probs[["buy","sell"]].idxmax(axis=1)
#     modelled_probs.loc[modelled_probs["bs_bool"],"signal_prob"] = modelled_probs[["buy","sell"]].max(axis=1)
#     print(modelled_probs)
    for cl in unique_classes:
        print(cl,"count:",str(len(modelled_probs.loc[modelled_probs["signal"] == cl,:])))
    return modelled_probs[["signal","signal_prob"]]

#Function to calculate the True Positive Rate for each classifier
def calc_tpr(probs_df,features,unique_classes):
    tpr_li = []
    for cl in unique_classes:
        true_pos = ((probs_df["real_signal"] == cl) & (probs_df["signal"] == cl)).sum()
        false_pos = ((probs_df["real_signal"] != cl) & (probs_df["signal"] == cl)).sum()
        true_neg = ((probs_df["real_signal"] != cl) & (probs_df["signal"] != cl)).sum()
        false_neg = ((probs_df["real_signal"] == cl) & (probs_df["signal"] != cl)).sum()
        #Calc tpr
        if sum([true_pos,false_neg]) != 0:
            tpr = true_pos / sum([true_pos,false_neg])
        else:
            tpr = None
        #calc fpr
        if sum([true_pos,false_neg]) != 0:
            fpr = false_pos / sum([false_pos,true_neg])
        else:
            fpr = None
        #calc accuracy
        if sum([true_pos,false_pos,true_neg,false_neg]) != 0:
            acc = sum([true_pos,true_neg])/sum([true_pos,false_pos,true_neg,false_neg])
        if sum([true_pos,false_pos]) != 0:
            pos_acc = true_pos/sum([true_pos,false_pos])
        if sum([true_neg,false_neg]) != 0:
            neg_acc = true_neg/sum([true_neg,false_neg])
        else:
            acc = None
        tpr_li.append({
            "features":features.copy()
            ,"feature_count":len(features)
            ,"signal":cl
            ,"true_pos":true_pos
            ,"false_pos":false_pos
            ,"true_neg":true_neg
            ,"false_neg":false_neg
            ,"tpr":tpr
            ,"fpr":fpr
            ,"acc":acc
            ,"pos_acc":pos_acc
            ,"neg_acc":neg_acc
        })
        print('true_pos:{}, false_pos:{}, acc:{},pos_acc:{},neg_acc:{}'.format(true_pos,false_pos,acc,pos_acc,neg_acc))
    tpr_df = pd.DataFrame(tpr_li,columns=["features","feature_count","signal","true_pos","false_pos","true_neg","false_neg","tpr","fpr","acc","pos_acc","neg_acc"])
    return tpr_df

# Comparing individual features
- Loop through the feaures to find the tpr of each one.
- Then eliminate features with a very low accuracy.
- Then order in descending order of combined buy,sell,hold accuracy and combine in that order to find the best feature combination.

In [93]:
tpr_single_df = pd.DataFrame([])
count = 0
for ft in feature_cols[:2]:
    print("")
    print("count: {}".format(count))
    count += 1
    print("TESTING:",ft)
    #Create a list of unique values
    unique_classes = df_model_rand["signal"].unique()
#     print("unique_classes:",unique_classes)
    #Remove rows with nan or inf
    tmp_df_train = df_train[[ft,"signal"]].replace([np.inf,-np.inf],np.nan).dropna(axis=0)
    tmp_df_test = df_test[[ft,"signal"]].replace([np.inf,-np.inf],np.nan).dropna(axis=0)
    #Build each model and add to the models dictionary
    models = build_models(unique_classes,tmp_df_train,[ft])
#     print(models)
    #Calculate the probabilities of each
    modelled_probs = calc_probs_df(models,unique_classes,tmp_df_test,[ft])
#     print("modelled_probs: {}".format(modelled_probs.head()))
    #Compare these results to the actual results
    modelled_probs["real_signal"] = tmp_df_test["signal"]
    #Calculate the True Positive Rate (TPR) for each of the unique classifiers
    tpr_single_df = tpr_single_df.append(calc_tpr(modelled_probs,[ft],unique_classes))
tpr_single_df


count: 0
TESTING: Open
hold count: 33207
buy count: 36326
sell count: 26
true_pos:13125, false_pos:20082, acc:0.5605888526287037,pos_acc:0.39524798988165144,neg_acc:0.7116252200704225
true_pos:14241, false_pos:22085, acc:0.5585617964605587,pos_acc:0.392033254418323,neg_acc:0.7405891734119701
true_pos:9, false_pos:17, acc:0.6679509481159879,pos_acc:0.34615384615384615,neg_acc:0.6680712755094703

count: 1
TESTING: Close
hold count: 32803
buy count: 36228
sell count: 528
true_pos:13049, false_pos:19754, acc:0.5642116764185799,pos_acc:0.3977989818004451,neg_acc:0.7127271737947546
true_pos:14216, false_pos:22012, acc:0.5592518581348208,pos_acc:0.39240366567296014,neg_acc:0.740601842128949
true_pos:159, false_pos:369, acc:0.6650469385701347,pos_acc:0.30113636363636365,neg_acc:0.6678303950399096


,features,feature_count,signal,true_pos,false_pos,true_neg,false_neg,tpr,fpr,acc,pos_acc,neg_acc
0,[Open],1,hold,13125,20082,25869,10483,0.555956,0.437031,0.560589,0.395248,0.711625
1,[Open],1,buy,14241,22085,24612,8621,0.622911,0.472943,0.558562,0.392033,0.740589
2,[Open],1,sell,9,17,46453,23080,0.000390,0.000366,0.667951,0.346154,0.668071
0,[Close],1,hold,13049,19754,26197,10559,0.552736,0.429893,0.564212,0.397799,0.712727
1,[Close],1,buy,14216,22012,24685,8646,0.621818,0.471379,0.559252,0.392404,0.740602
2,[Close],1,sell,159,369,46101,22930,0.006886,0.007941,0.665047,0.301136,0.667830


In [91]:
# tpr_single_df.loc[tpr_single_df["signal"] == "buy",:]
tpr_single_df.loc[(tpr_single_df["signal"] == "buy") & (tpr_single_df["true_pos"] > 0),:].sort_values("pos_acc",ascending=False)

,features,feature_count,signal,true_pos,false_pos,true_neg,false_neg,tpr,fpr,acc,pos_acc,neg_acc
1,[Close],1,buy,14216,22012,24685,8646,0.621818,0.471379,0.559252,0.392404,0.740602
1,[Open],1,buy,14241,22085,24612,8621,0.622911,0.472943,0.558562,0.392033,0.740589


# Paired feature testing
Check all the pairings of features to see if this provides as better outcome.

In [12]:
#Create pairings
ft_li = []
for ft1 in feature_cols:
    for ft2 in feature_cols:
        #Skip if identical
        if ft1 == ft2:
            continue
        if ft1 > ft2:
            if [ft1,ft2] not in ft_li:
                ft_li.append([ft1,ft2])
        else:
            if [ft2,ft1] not in ft_li:
                ft_li.append([ft2,ft1])
print(len(ft_li))
ft_li

1326


[['Open', 'Close'],
 ['Open', 'High'],
 ['Open', 'Low'],
 ['Volume', 'Open'],
 ['change_price', 'Open'],
 ['per_change_price', 'Open'],
 ['close_shift1', 'Open'],
 ['change_close_shift1', 'Open'],
 ['vol_shift1', 'Open'],
 ['change_vol_shift1', 'Open'],
 ['Open', 'EMA26_shift1'],
 ['change_EMA26_shift1', 'Open'],
 ['prev_max_close', 'Open'],
 ['prev_min_close', 'Open'],
 ['prev_max_EMA26', 'Open'],
 ['prev_min_EMA26', 'Open'],
 ['prev_max_close_date_change', 'Open'],
 ['prev_min_close_date_change', 'Open'],
 ['prev_max_MACD_date_change', 'Open'],
 ['prev_min_MACD_date_change', 'Open'],
 ['prev_max_EMA26_date_change', 'Open'],
 ['prev_min_EMA26_date_change', 'Open'],
 ['prev_max_signal_date_change', 'Open'],
 ['prev_min_signal_date_change', 'Open'],
 ['close_13_norm', 'Open'],
 ['close_26_norm', 'Open'],
 ['close_52_norm', 'Open'],
 ['Open', 'MACD_pos_val'],
 ['Open', 'MACD_neg_val'],
 ['signal_pos_val', 'Open'],
 ['signal_neg_val', 'Open'],
 ['change_price_pos_val', 'Open'],
 ['change_

In [13]:
#Create pairings
tpr_pair_df = pd.DataFrame([])
count = 0
print("TOTAL TO RUN: {}".format(len(ft_li)))
for ft_pair in ft_li:
    print("")
    print("count: {}".format(count))
    count += 1
    print("TESTING: {},{}".format(ft_pair[0],ft_pair[1]))
    #Create a list of unique values
    unique_classes = df_model_rand["signal"].unique()
#     print("unique_classes:",unique_classes)
    #Remove rows with nan or inf
    tmp_df_train = df_train[[ft_pair[0],ft_pair[1],"signal"]].replace([np.inf,-np.inf],np.nan).dropna(axis=0)
    tmp_df_test = df_test[[ft_pair[0],ft_pair[1],"signal"]].replace([np.inf,-np.inf],np.nan).dropna(axis=0)
    #Build each model and add to the models dictionary
    models = build_models(unique_classes,tmp_df_train,[ft_pair[0],ft_pair[1]])
#     print(models)
    #Calculate the probabilities of each
    modelled_probs = calc_probs_df(models,unique_classes,tmp_df_test,[ft_pair[0],ft_pair[1]])
#     print("modelled_probs: {}".format(modelled_probs.head()))
    #Compare these results to the actual results
    modelled_probs["real_signal"] = tmp_df_test["signal"]
    #Calculate the True Positive Rate (TPR) for each of the unique classifiers
    tpr_pair_df = tpr_pair_df.append(calc_tpr(modelled_probs,[ft_pair[0],ft_pair[1]],unique_classes))
tpr_pair_df

TOTAL TO RUN: 1326

count: 0
TESTING: Open,Close
hold count: 11672
buy count: 34400
sell count: 23487
true_pos:4709, false_pos:6963, acc:0.6282005204215126
true_pos:13596, false_pos:20804, acc:0.567705113644532
true_pos:8552, false_pos:14935, acc:0.5763021320030478

count: 1
TESTING: Open,High
hold count: 4028
buy count: 35836
sell count: 29695
true_pos:1356, false_pos:2672, acc:0.6416854756393853
true_pos:14102, false_pos:21734, acc:0.5616095688552164
true_pos:10366, false_pos:19329, acc:0.5392113170114579

count: 2
TESTING: Open,Low
hold count: 6897
buy count: 34933
sell count: 27729
true_pos:2619, false_pos:4278, acc:0.6367544099253871
true_pos:13842, false_pos:21091, acc:0.5671156859644331
true_pos:9699, false_pos:18030, acc:0.5482971290559093

count: 3
TESTING: Volume,Open
hold count: 3780
buy count: 35806
sell count: 29973
true_pos:1330, false_pos:2450, acc:0.6445032274759557
true_pos:14088, false_pos:21718, acc:0.5616383214249774
true_pos:10532, false_pos:19441, acc:0.5399876363

true_pos:13588, false_pos:20820, acc:0.5673600828074009
true_pos:9415, false_pos:16724, acc:0.5629896922037407

count: 32
TESTING: change_price_neg_val,Open
hold count: 18577
buy count: 34056
sell count: 16926
true_pos:7551, false_pos:11026, acc:0.6106470765824695
true_pos:13454, false_pos:20602, acc:0.5685676907373597
true_pos:6652, false_pos:10274, acc:0.6159950545580011

count: 33
TESTING: per_change_price_pos_val,Open
hold count: 7251
buy count: 34058
sell count: 28250
true_pos:3130, false_pos:4121, acc:0.6463577682255351
true_pos:13478, false_pos:20580, acc:0.5692289998418608
true_pos:10091, false_pos:18159, acc:0.5520780919794707

count: 34
TESTING: per_change_price_neg_val,Open
hold count: 14697
buy count: 33215
sell count: 21647
true_pos:5801, false_pos:8896, acc:0.6161100648370448
true_pos:13130, false_pos:20085, acc:0.5713423137192887
true_pos:8256, false_pos:13391, acc:0.5942437355338633

count: 35
TESTING: max_change_close_pos_val,Open
hold count: 3745
buy count: 32188
sell


count: 63
TESTING: prev_min_close,Close
hold count: 14971
buy count: 31298
sell count: 23290
true_pos:7700, false_pos:7271, acc:0.6667720927557901
true_pos:12667, false_pos:18631, acc:0.5855892120358257
true_pos:10524, false_pos:12766, acc:0.6358343276930376

count: 64
TESTING: prev_max_EMA26,Close
hold count: 14175
buy count: 32188
sell count: 23196
true_pos:6752, false_pos:7423, acc:0.6509581793872827
true_pos:12681, false_pos:19507, acc:0.5731968544688681
true_pos:9723, false_pos:13473, acc:0.614154890093302

count: 65
TESTING: prev_min_EMA26,Close
hold count: 13061
buy count: 32371
sell count: 24127
true_pos:5842, false_pos:7219, acc:0.6408085222616772
true_pos:12915, false_pos:19456, acc:0.5772940956597996
true_pos:9642, false_pos:14485, acc:0.598441610718958

count: 66
TESTING: prev_max_close_date_change,Close
hold count: 8344
buy count: 27987
sell count: 33228
true_pos:3442, false_pos:4902, acc:0.6396152906165988
true_pos:12364, false_pos:15623, acc:0.6244770626374733
true_pos:

hold count: 10900
buy count: 34281
sell count: 24378
true_pos:4185, false_pos:6715, acc:0.6242326657945054
true_pos:13523, false_pos:20758, acc:0.5673169539527595
true_pos:8824, false_pos:15554, acc:0.5713135611495277

count: 95
TESTING: min_change_EMA26_pos_val,Close
hold count: 3013
buy count: 36040
sell count: 30506
true_pos:1107, false_pos:1906, acc:0.6491180149225837
true_pos:14136, false_pos:21904, acc:0.5596543941114737
true_pos:10562, false_pos:19944, acc:0.5331876536465446

count: 96
TESTING: min_change_EMA26_neg_val,Close
hold count: 12684
buy count: 33050
sell count: 23825
true_pos:4808, false_pos:7876, acc:0.6164982245288173
true_pos:13038, false_pos:20012, acc:0.5710691643065599
true_pos:9101, false_pos:14724, acc:0.5872281085121983

count: 97
TESTING: max_change_signal_pos_val,Close
hold count: 4858
buy count: 33551
sell count: 31150
true_pos:2529, false_pos:2329, acc:0.6634799235181644
true_pos:13293, false_pos:20258, acc:0.5711985508704841
true_pos:11453, false_pos:1969

true_pos:10624, false_pos:16161, acc:0.5917278856797826
true_pos:11511, false_pos:13757, acc:0.6357768225535158

count: 126
TESTING: MACD_pos_val,High
hold count: 11181
buy count: 31173
sell count: 27205
true_pos:5769, false_pos:5412, acc:0.6657370002443969
true_pos:12364, false_pos:18809, acc:0.5786742190083238
true_pos:11114, false_pos:16091, acc:0.5965151885449762

count: 127
TESTING: MACD_neg_val,High
hold count: 21953
buy count: 32605
sell count: 15001
true_pos:9905, false_pos:12048, acc:0.6297962880432438
true_pos:12932, false_pos:19673, acc:0.5744188386837074
true_pos:6783, false_pos:8218, acc:0.6474359895915698

count: 128
TESTING: signal_pos_val,High
hold count: 12610
buy count: 33623
sell count: 23326
true_pos:5634, false_pos:6976, acc:0.6413116922324933
true_pos:13312, false_pos:20311, acc:0.5707097571845484
true_pos:8981, false_pos:14345, acc:0.5909515662962377

count: 129
TESTING: signal_neg_val,High
hold count: 17269
buy count: 32526
sell count: 19764
true_pos:6984, false

...

hold count: 42471
buy count: 8007
sell count: 19081
true_pos:16333, false_pos:26138, acc:0.5196451932891503
true_pos:2731, false_pos:5276, acc:0.6347417300421225
true_pos:8305, false_pos:10776, acc:0.632542158455412

count: 1227
TESTING: max_change_close_neg_val,max_change_EMA26_pos_val
hold count: 16520
buy count: 34266
sell count: 18773
true_pos:7495, false_pos:9025, acc:0.6386089506749666
true_pos:11724, false_pos:22542, acc:0.5158067252260671
true_pos:8053, false_pos:10720, acc:0.6297244066188415

count: 1228
TESTING: max_change_close_neg_val,max_change_EMA26_neg_val
hold count: 41109
buy count: 9734
sell count: 18716
true_pos:15789, false_pos:25320, acc:0.5235842953463966
true_pos:3418, false_pos:6316, acc:0.6296669014793197
true_pos:7924, false_pos:10792, acc:0.6268347733578689

count: 1229
TESTING: min_change_EMA26_pos_val,max_change_close_neg_val
hold count: 16332
buy count: 35726
sell count: 17501
true_pos:7027, false_pos:9305, acc:0.6278554895843816
true_pos:12643, false_pos:

hold count: 56615
buy count: 3937
sell count: 9007
true_pos:21352, false_pos:35263, acc:0.4606161675699766
true_pos:1370, false_pos:2567, acc:0.6541209620609842
true_pos:5193, false_pos:3814, acc:0.6878908552451876

count: 1256
TESTING: min_change_close_neg_val,max_change_signal_pos_val
hold count: 8790
buy count: 54524
sell count: 6245
true_pos:3983, false_pos:4807, acc:0.6487586078005722
true_pos:18702, false_pos:35822, acc:0.42520737790940066
true_pos:4265, false_pos:1980, acc:0.7009157693468854

count: 1257
TESTING: min_change_close_neg_val,max_change_signal_neg_val
hold count: 7984
buy count: 44841
sell count: 16734
true_pos:2716, false_pos:5268, acc:0.6239163875271352
true_pos:15849, false_pos:28992, acc:0.4823818628789948
true_pos:7692, false_pos:9042, acc:0.648657973806409

count: 1258
TESTING: min_change_signal_pos_val,min_change_close_neg_val
hold count: 20936
buy count: 43162
sell count: 5461
true_pos:8230, false_pos:12706, acc:0.5962564154171279
true_pos:15337, false_pos:27

hold count: 16972
buy count: 36700
sell count: 15887
true_pos:7395, false_pos:9577, acc:0.6292356129329059
true_pos:13631, false_pos:23069, acc:0.5356459983611035
true_pos:5521, false_pos:10366, acc:0.5984128581491971

count: 1285
TESTING: min_change_MACD_pos_val,min_change_EMA26_neg_val
hold count: 28769
buy count: 32757
sell count: 8033
true_pos:12453, false_pos:16316, acc:0.6050690780488506
true_pos:11605, false_pos:21152, acc:0.5340789833091333
true_pos:3890, false_pos:4143, acc:0.6644287583202749

count: 1286
TESTING: min_change_MACD_pos_val,max_change_signal_pos_val
hold count: 14775
buy count: 6702
sell count: 48082
true_pos:6493, false_pos:8282, acc:0.6348854928909271
true_pos:2690, false_pos:4012, acc:0.6523239264509265
true_pos:17592, false_pos:30490, acc:0.4826406360068431

count: 1287
TESTING: min_change_MACD_pos_val,max_change_signal_neg_val
hold count: 19761
buy count: 30894
sell count: 18904
true_pos:8808, false_pos:10953, acc:0.629767535473483
true_pos:11244, false_pos:

true_pos:14056, false_pos:25322, acc:0.5093661495996205
true_pos:9080, false_pos:16691, acc:0.5586480541698414

count: 1313
TESTING: min_change_EMA26_pos_val,max_change_signal_neg_val
hold count: 12910
buy count: 37596
sell count: 19053
true_pos:5434, false_pos:7476, acc:0.6312482928161705
true_pos:13634, false_pos:23962, acc:0.5228511048174931
true_pos:7642, false_pos:11411, acc:0.6138817406805733

count: 1314
TESTING: min_change_signal_pos_val,min_change_EMA26_pos_val
hold count: 12975
buy count: 41157
sell count: 15427
true_pos:5440, false_pos:7535, acc:0.6304863497175061
true_pos:14565, false_pos:26592, acc:0.4984257968055895
true_pos:5247, false_pos:10180, acc:0.5971477450797165

count: 1315
TESTING: min_change_signal_neg_val,min_change_EMA26_pos_val
hold count: 18299
buy count: 40871
sell count: 10389
true_pos:7441, false_pos:10858, acc:0.6114809011055363
true_pos:14506, false_pos:26365, acc:0.500841012665507
true_pos:4703, false_pos:5686, acc:0.6539340703575383

count: 1316
TEST

,features,feature_count,signal,true_pos,false_pos,true_neg,false_neg,tpr,fpr,acc,pos_acc,neg_acc
0,"[Open, Close]",2,hold,4709,6963,38988,18899,0.199466,0.151531,0.628201,0.403444,0.673519
1,"[Open, Close]",2,buy,13596,20804,25893,9266,0.594699,0.445510,0.567705,0.395233,0.736454
2,"[Open, Close]",2,sell,8552,14935,31535,14537,0.370393,0.321390,0.576302,0.364116,0.684472
0,"[Open, High]",2,hold,1356,2672,43279,22252,0.057438,0.058149,0.641685,0.336643,0.660436
1,"[Open, High]",2,buy,14102,21734,24963,8760,0.616831,0.465426,0.561610,0.393515,0.740237
2,"[Open, High]",2,sell,10366,19329,27141,12723,0.448958,0.415946,0.539211,0.349082,0.680840
0,"[Open, Low]",2,hold,2619,4278,41673,20989,0.110937,0.093099,0.636754,0.379730,0.665044
1,"[Open, Low]",2,buy,13842,21091,25606,9020,0.605459,0.451656,0.567116,0.396244,0.739502
2,"[Open, Low]",2,sell,9699,18030,28440,13390,0.420070,0.387992,0.548297,0.349778,0.679895
0,"[Volume, Open]",2,hold,1330,2450,43501,22278,0.056337,0.053318,0.644503,0.351852,0.661320


In [14]:
# tpr_pair_df["ppv"] = tpr_pair_df["true_pos"] / sum(["true_pos","false_pos"])
tpr_pair_df.loc[(tpr_pair_df["signal"] == "buy") & (tpr_pair_df["true_pos"] > 0),:].sort_values("pos_acc",ascending=False)

,features,feature_count,signal,true_pos,false_pos,true_neg,false_neg,tpr,fpr,acc,pos_acc,neg_acc
1,"[max_change_signal_pos_val, max_change_close_n...",2,buy,7,6,46691,22855,0.000306,0.000128,0.671344,0.538462,0.671369
1,"[min_change_EMA26_neg_val, change_close_shift1]",2,buy,15,15,46682,22847,0.000656,0.000321,0.671329,0.500000,0.671403
1,"[min_change_close_pos_val, max_change_close_po...",2,buy,2851,3543,43154,20011,0.124705,0.075872,0.661381,0.445887,0.683195
1,"[min_change_EMA26_pos_val, max_change_close_po...",2,buy,4078,5073,41624,18784,0.178375,0.108637,0.657025,0.445634,0.689048
1,"[prev_max_close_date_change, Low]",2,buy,12459,15609,31088,10403,0.544965,0.334261,0.626044,0.443886,0.749271
1,"[prev_max_close_date_change, close_shift1]",2,buy,12467,15753,30944,10395,0.545315,0.337345,0.624089,0.441779,0.748543
1,"[prev_max_close_date_change, Close]",2,buy,12364,15623,31074,10498,0.540810,0.334561,0.624477,0.441777,0.747474
1,"[prev_max_close_date_change, Open]",2,buy,12441,15743,30954,10421,0.544178,0.337131,0.623859,0.441421,0.748133
1,"[signal_pos_val, max_change_close_pos_val]",2,buy,3587,4542,42155,19275,0.156898,0.097265,0.657600,0.441260,0.686228
1,"[prev_max_close_date_change, High]",2,buy,12340,15631,31066,10522,0.539760,0.334732,0.624017,0.441171,0.746994


# Cumulative feature testing

In [15]:
#Put the features into an ordered list
tpr_pair_df = tpr_pair_df.sort_values("pos_acc",ascending=False).reset_index(drop=True)
ft_li = []
for row in tpr_pair_df.iterrows():
    for ft in row[1].features:
        if ft not in ft_li:
            ft_li.append(ft)
ft_li

['signal_pos_val',
 'min_change_close_neg_val',
 'min_change_signal_pos_val',
 'max_change_EMA26_pos_val',
 'change_vol_shift1',
 'min_change_MACD_pos_val',
 'prev_min_close_date_change',
 'vol_shift1',
 'change_close_shift1',
 'Volume',
 'max_change_EMA26_neg_val',
 'change_price_pos_val',
 'max_change_MACD_pos_val',
 'max_change_signal_pos_val',
 'per_change_price_pos_val',
 'min_change_EMA26_pos_val',
 'per_change_price_neg_val',
 'change_price',
 'min_change_EMA26_neg_val',
 'min_change_MACD_neg_val',
 'prev_max_EMA26',
 'min_change_close_pos_val',
 'prev_min_EMA26',
 'min_change_signal_neg_val',
 'EMA26_shift1',
 'prev_max_close_date_change',
 'prev_min_close',
 'max_change_close_pos_val',
 'Low',
 'prev_max_close',
 'Close',
 'High',
 'close_shift1',
 'Open',
 'close_26_norm',
 'prev_min_MACD_date_change',
 'max_change_close_neg_val',
 'close_13_norm',
 'prev_max_MACD_date_change',
 'MACD_pos_val',
 'close_52_norm',
 'MACD_neg_val',
 'per_change_price',
 'max_change_signal_neg_va

In [16]:
#Itterate through the rows adding a new feature each time and add to the dataframe of results
tpr_cum_df = pd.DataFrame([])
count = 0
print("TOTAL TO RUN: {}".format(len(ft_li)))
cum_ft_li = []
for ft in ft_li:
    print("")
    print("count: {}".format(count))
    count += 1
    cum_ft_li.append(ft)
    print("TESTING: {}".format(cum_ft_li))
    #Create a list of unique values
    unique_classes = df_model_rand["signal"].unique()
#     print("unique_classes:",unique_classes)
    #Remove rows with nan or inf
    tmp_li = cum_ft_li.copy()
    tmp_li.append("signal")
    tmp_df_train = df_train[tmp_li].replace([np.inf,-np.inf],np.nan).dropna(axis=0)
    tmp_df_test = df_test[tmp_li].replace([np.inf,-np.inf],np.nan).dropna(axis=0)
    #Build each model and add to the models dictionary
    models = build_models(unique_classes,tmp_df_train,cum_ft_li)
#     print(models)
    #Calculate the probabilities of each
    modelled_probs = calc_probs_df(models,unique_classes,tmp_df_test,cum_ft_li)
#     print("modelled_probs: {}".format(modelled_probs.head()))
    #Compare these results to the actual results
    modelled_probs["real_signal"] = tmp_df_test["signal"]
    #Calculate the True Positive Rate (TPR) for each of the unique classifiers
    tpr_cum_df = tpr_cum_df.append(calc_tpr(modelled_probs,cum_ft_li,unique_classes))
tpr_cum_df

TOTAL TO RUN: 52

count: 0
TESTING: ['signal_pos_val']
hold count: 25980
buy count: 0
sell count: 43579
true_pos:10899, false_pos:15081, acc:0.6004830431719835
true_pos:0, false_pos:0, acc:0.6713293750628962
true_pos:15727, false_pos:27852, acc:0.4937535042194396

count: 1
TESTING: ['signal_pos_val', 'min_change_close_neg_val']
hold count: 24038
buy count: 40248
sell count: 5273
true_pos:10349, false_pos:13689, acc:0.6125878750413318
true_pos:14760, false_pos:25488, acc:0.5171005908653086
true_pos:3634, false_pos:1639, acc:0.6967466467315516

count: 2
TESTING: ['signal_pos_val', 'min_change_close_neg_val', 'min_change_signal_pos_val']
hold count: 24013
buy count: 40266
sell count: 5280
true_pos:10359, false_pos:13654, acc:0.6132348078609525
true_pos:14772, false_pos:25494, acc:0.5171868485745914
true_pos:3639, false_pos:1641, acc:0.696789775586193

count: 3
TESTING: ['signal_pos_val', 'min_change_close_neg_val', 'min_change_signal_pos_val', 'max_change_EMA26_pos_val']
hold count: 22897

hold count: 20252
buy count: 29411
sell count: 19896
true_pos:10086, false_pos:10166, acc:0.6594545637516352
true_pos:12493, false_pos:16918, acc:0.6077143144668554
true_pos:9590, false_pos:10306, acc:0.6577725384206213

count: 18
TESTING: ['signal_pos_val', 'min_change_close_neg_val', 'min_change_signal_pos_val', 'max_change_EMA26_pos_val', 'change_vol_shift1', 'min_change_MACD_pos_val', 'prev_min_close_date_change', 'vol_shift1', 'change_close_shift1', 'Volume', 'max_change_EMA26_neg_val', 'change_price_pos_val', 'max_change_MACD_pos_val', 'max_change_signal_pos_val', 'per_change_price_pos_val', 'min_change_EMA26_pos_val', 'per_change_price_neg_val', 'change_price', 'min_change_EMA26_neg_val']
hold count: 20752
buy count: 29027
sell count: 19780
true_pos:10342, false_pos:10410, acc:0.6596270791702008
true_pos:12183, false_pos:16844, acc:0.6043215112350666
true_pos:9666, false_pos:10114, acc:0.661625382768585

count: 19
TESTING: ['signal_pos_val', 'min_change_close_neg_val', 'min_chan

true_pos:12751, false_pos:14010, acc:0.6532296323983956
true_pos:14133, false_pos:14240, acc:0.6665276959128222

count: 28
TESTING: ['signal_pos_val', 'min_change_close_neg_val', 'min_change_signal_pos_val', 'max_change_EMA26_pos_val', 'change_vol_shift1', 'min_change_MACD_pos_val', 'prev_min_close_date_change', 'vol_shift1', 'change_close_shift1', 'Volume', 'max_change_EMA26_neg_val', 'change_price_pos_val', 'max_change_MACD_pos_val', 'max_change_signal_pos_val', 'per_change_price_pos_val', 'min_change_EMA26_pos_val', 'per_change_price_neg_val', 'change_price', 'min_change_EMA26_neg_val', 'min_change_MACD_neg_val', 'prev_max_EMA26', 'min_change_close_pos_val', 'prev_min_EMA26', 'min_change_signal_neg_val', 'EMA26_shift1', 'prev_max_close_date_change', 'prev_min_close', 'max_change_close_pos_val', 'Low']
hold count: 14723
buy count: 26606
sell count: 28230
true_pos:8500, false_pos:6223, acc:0.6933394672148824
true_pos:12724, false_pos:13882, acc:0.6546816371713222
true_pos:14060, false

hold count: 15204
buy count: 26583
sell count: 27772
true_pos:8857, false_pos:6347, acc:0.6966891415920298
true_pos:12639, false_pos:13944, acc:0.6525683232938944
true_pos:14215, false_pos:13557, acc:0.677525553846375

count: 37
TESTING: ['signal_pos_val', 'min_change_close_neg_val', 'min_change_signal_pos_val', 'max_change_EMA26_pos_val', 'change_vol_shift1', 'min_change_MACD_pos_val', 'prev_min_close_date_change', 'vol_shift1', 'change_close_shift1', 'Volume', 'max_change_EMA26_neg_val', 'change_price_pos_val', 'max_change_MACD_pos_val', 'max_change_signal_pos_val', 'per_change_price_pos_val', 'min_change_EMA26_pos_val', 'per_change_price_neg_val', 'change_price', 'min_change_EMA26_neg_val', 'min_change_MACD_neg_val', 'prev_max_EMA26', 'min_change_close_pos_val', 'prev_min_EMA26', 'min_change_signal_neg_val', 'EMA26_shift1', 'prev_max_close_date_change', 'prev_min_close', 'max_change_close_pos_val', 'Low', 'prev_max_close', 'Close', 'High', 'close_shift1', 'Open', 'close_26_norm', 'p

hold count: 15210
buy count: 26553
sell count: 27796
true_pos:8942, false_pos:6268, acc:0.6990468523124255
true_pos:12844, false_pos:13709, acc:0.6588938886412973
true_pos:14390, false_pos:13406, acc:0.6822122227174053

count: 45
TESTING: ['signal_pos_val', 'min_change_close_neg_val', 'min_change_signal_pos_val', 'max_change_EMA26_pos_val', 'change_vol_shift1', 'min_change_MACD_pos_val', 'prev_min_close_date_change', 'vol_shift1', 'change_close_shift1', 'Volume', 'max_change_EMA26_neg_val', 'change_price_pos_val', 'max_change_MACD_pos_val', 'max_change_signal_pos_val', 'per_change_price_pos_val', 'min_change_EMA26_pos_val', 'per_change_price_neg_val', 'change_price', 'min_change_EMA26_neg_val', 'min_change_MACD_neg_val', 'prev_max_EMA26', 'min_change_close_pos_val', 'prev_min_EMA26', 'min_change_signal_neg_val', 'EMA26_shift1', 'prev_max_close_date_change', 'prev_min_close', 'max_change_close_pos_val', 'Low', 'prev_max_close', 'Close', 'High', 'close_shift1', 'Open', 'close_26_norm', '

hold count: 15094
buy count: 26649
sell count: 27816
true_pos:8904, false_pos:6190, acc:0.6996219037076439
true_pos:12904, false_pos:13745, acc:0.6592389194784284
true_pos:14609, false_pos:13207, acc:0.6882215097974381


,features,feature_count,signal,true_pos,false_pos,true_neg,false_neg,tpr,fpr,acc,pos_acc,neg_acc
0,[signal_pos_val],1,hold,10899,15081,30870,12709,0.461666,0.328197,0.600483,0.419515,0.708369
1,[signal_pos_val],1,buy,0,0,46697,22862,0.000000,0.000000,0.671329,0.419515,0.671329
2,[signal_pos_val],1,sell,15727,27852,18618,7362,0.681147,0.599354,0.493754,0.360885,0.716628
0,"[signal_pos_val, min_change_close_neg_val]",2,hold,10349,13689,32262,13259,0.438368,0.297904,0.612588,0.430527,0.708728
1,"[signal_pos_val, min_change_close_neg_val]",2,buy,14760,25488,21209,8102,0.645613,0.545817,0.517101,0.366726,0.723585
2,"[signal_pos_val, min_change_close_neg_val]",2,sell,3634,1639,44831,19455,0.157391,0.035270,0.696747,0.689171,0.697368
0,"[signal_pos_val, min_change_close_neg_val, min...",3,hold,10359,13654,32297,13249,0.438792,0.297143,0.613235,0.431391,0.709107
1,"[signal_pos_val, min_change_close_neg_val, min...",3,buy,14772,25494,21203,8090,0.646138,0.545945,0.517187,0.366860,0.723825
2,"[signal_pos_val, min_change_close_neg_val, min...",3,sell,3639,1641,44829,19450,0.157608,0.035313,0.696790,0.689205,0.697413
0,"[signal_pos_val, min_change_close_neg_val, min...",4,hold,9937,12960,32991,13671,0.420917,0.282040,0.617145,0.433987,0.707021


In [19]:
tpr_cum_df.loc[(tpr_cum_df["signal"] == "buy"),:].sort_values("pos_acc",ascending=False)

,features,feature_count,signal,true_pos,false_pos,true_neg,false_neg,tpr,fpr,acc,pos_acc,neg_acc
1,"[signal_pos_val, min_change_close_neg_val, min...",50,buy,12905,13667,33030,9957,0.564474,0.292674,0.660375,0.485662,0.768372
1,"[signal_pos_val, min_change_close_neg_val, min...",49,buy,12916,13698,32999,9946,0.564955,0.293338,0.660087,0.485308,0.768401
1,"[signal_pos_val, min_change_close_neg_val, min...",51,buy,12938,13738,32959,9924,0.565917,0.294194,0.659828,0.485005,0.768580
1,"[signal_pos_val, min_change_close_neg_val, min...",52,buy,12904,13745,32952,9958,0.564430,0.294344,0.659239,0.484221,0.767933
1,"[signal_pos_val, min_change_close_neg_val, min...",45,buy,12844,13709,32988,10018,0.561806,0.293573,0.658894,0.483712,0.767056
1,"[signal_pos_val, min_change_close_neg_val, min...",48,buy,12852,13779,32918,10010,0.562156,0.295072,0.658003,0.482595,0.766819
1,"[signal_pos_val, min_change_close_neg_val, min...",43,buy,12815,13756,32941,10047,0.560537,0.294580,0.657801,0.482293,0.766284
1,"[signal_pos_val, min_change_close_neg_val, min...",44,buy,12821,13774,32923,10041,0.560800,0.294965,0.657629,0.482083,0.766293
1,"[signal_pos_val, min_change_close_neg_val, min...",47,buy,12846,13803,32894,10016,0.561893,0.295586,0.657571,0.482044,0.766581
1,"[signal_pos_val, min_change_close_neg_val, min...",46,buy,12793,13752,32945,10069,0.559575,0.294494,0.657543,0.481936,0.765913


In [45]:
#Convert features into string
tmp_df = tpr_cum_df.copy()
def ft_str(li_in):
    str_out = ''
    for i in li_in:
        str_out += i+','
    str_out = str_out[:-1]
    return str_out
tmp_df["ft_str"] = [ft_str(x) for x in tmp_df["features"]]
tmp_df
#Average the model over all the signals
tpr_cum_sum_df = tmp_df.loc[(tmp_df["signal"] == "buy") & (tmp_df["tpr"] > 0.5),:][["ft_str","feature_count","true_pos","false_pos","true_neg","false_neg","tpr","fpr","acc"]].groupby("ft_str").mean().reset_index()
tpr_cum_sum_df.sort_values("acc",ascending=False,inplace=True)
tpr_cum_sum_df.reset_index(drop=True)

,ft_str,feature_count,true_pos,false_pos,true_neg,false_neg,tpr,fpr,acc
0,"signal_pos_val,min_change_close_neg_val,min_ch...",50,12905,13667,33030,9957,0.564474,0.292674,0.660375
1,"signal_pos_val,min_change_close_neg_val,min_ch...",49,12916,13698,32999,9946,0.564955,0.293338,0.660087
2,"signal_pos_val,min_change_close_neg_val,min_ch...",51,12938,13738,32959,9924,0.565917,0.294194,0.659828
3,"signal_pos_val,min_change_close_neg_val,min_ch...",52,12904,13745,32952,9958,0.564430,0.294344,0.659239
4,"signal_pos_val,min_change_close_neg_val,min_ch...",45,12844,13709,32988,10018,0.561806,0.293573,0.658894
5,"signal_pos_val,min_change_close_neg_val,min_ch...",48,12852,13779,32918,10010,0.562156,0.295072,0.658003
6,"signal_pos_val,min_change_close_neg_val,min_ch...",43,12815,13756,32941,10047,0.560537,0.294580,0.657801
7,"signal_pos_val,min_change_close_neg_val,min_ch...",44,12821,13774,32923,10041,0.560800,0.294965,0.657629
8,"signal_pos_val,min_change_close_neg_val,min_ch...",47,12846,13803,32894,10016,0.561893,0.295586,0.657571
9,"signal_pos_val,min_change_close_neg_val,min_ch...",46,12793,13752,32945,10069,0.559575,0.294494,0.657543


# Train the final model

In [51]:
#Get the most effective model
fm_features = tpr_cum_sum_df.iloc[0,:]
ft_li = fm_features.ft_str.split(",")
print("BUILDING: {}".format(cum_ft_li))

#Create a list of unique values
unique_classes = df_model_rand["signal"].unique()
print("UNIQUE CLASSES:",unique_classes)
#Remove rows with nan or inf
tmp_li = cum_ft_li.copy()
tmp_li.append("signal")
tmp_df_train = df_train[tmp_li].replace([np.inf,-np.inf],np.nan).dropna(axis=0)
tmp_df_test = df_test[tmp_li].replace([np.inf,-np.inf],np.nan).dropna(axis=0)
#Build each model and add to the models dictionary
final_models = build_models(unique_classes,tmp_df_train,ft_li)
final_models

BUILDING: ['signal_pos_val', 'min_change_close_neg_val', 'min_change_signal_pos_val', 'max_change_EMA26_pos_val', 'change_vol_shift1', 'min_change_MACD_pos_val', 'prev_min_close_date_change', 'vol_shift1', 'change_close_shift1', 'Volume', 'max_change_EMA26_neg_val', 'change_price_pos_val', 'max_change_MACD_pos_val', 'max_change_signal_pos_val', 'per_change_price_pos_val', 'min_change_EMA26_pos_val', 'per_change_price_neg_val', 'change_price', 'min_change_EMA26_neg_val', 'min_change_MACD_neg_val', 'prev_max_EMA26', 'min_change_close_pos_val', 'prev_min_EMA26', 'min_change_signal_neg_val', 'EMA26_shift1', 'prev_max_close_date_change', 'prev_min_close', 'max_change_close_pos_val', 'Low', 'prev_max_close', 'Close', 'High', 'close_shift1', 'Open', 'close_26_norm', 'prev_min_MACD_date_change', 'max_change_close_neg_val', 'close_13_norm', 'prev_max_MACD_date_change', 'MACD_pos_val', 'close_52_norm', 'MACD_neg_val', 'per_change_price', 'max_change_signal_neg_val', 'prev_max_signal_date_change'

{'hold': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
           penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
           verbose=0, warm_start=False),
 'buy': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
           penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
           verbose=0, warm_start=False),
 'sell': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
           penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
           verbose=0, warm_start=False)}

# Exporting the model

In [56]:
#Import the modules
from sklearn.externals import joblib as jl

In [57]:
#Export the model
jl.dump(final_models,path+r'log_reg_model.joblib')

['C:\\Users\\Robert\\Documents\\Python Scripts\\Stock trading - ML modelling\\Historical prices\\log_reg_model.joblib']

In [72]:
#Export a list of the features for this model
file_object = open(path+r'log_reg_model_feature_list.txt','w')
file_object.write(fm_features.ft_str)
file_object.close()